In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [5]:
data=pd.read_csv('SMS.csv',encoding='unicode_escape')

In [6]:
data.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [7]:
data=data.drop(['S. No.'],axis=1)

In [8]:
data.columns

Index(['Message_body', 'Label'], dtype='object')

In [9]:
lm=WordNetLemmatizer()
corpus=[]

In [10]:
for i in range(len(data)):
    review=re.sub('[^a-zA-Z]',' ',data['Message_body'][i])
    review=review.lower()
    review=review.split()
    review=[lm.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

In [13]:
print(corpus)

['rofl true name', 'guy bitching acted like interested buying something else next week gave u free', 'pity mood suggestion', 'b going esplanade fr home', 'nd time tried contact u u pound prize claim easy call p per minute bt national rate', 'reminder get pound free call credit detail great offer pls reply text valid name house postcode', 'huh lei', 'wait til least wednesday see get', 'ard like dat lor', 'ok lor sony ericsson salesman ask shuhui say quite gd use considering', 'get dump heap mom decided come lowes boring', 'anything lor juz u lor', 'next lt gt hour imma flip shit', 'sorry call later', 'meant calculation lt gt unit lt gt school really expensive started practicing accent important decided year dental school nmde exam', 'yes u texted pshew missing much', 'yeh indian nice tho kane bit shud go drink sometime soon mite hav go da work laugh soon love pete x x', 'well keep mind got enough gas one round trip barring sudden influx cash', 'hahaha use brain dear', 'safe trip nigeria

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tf=TfidfVectorizer()
x=tf.fit_transform(corpus).toarray()

In [19]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
y=pd.get_dummies(data.Label,drop_first=True)

In [22]:
y

,Spam
0,0
1,0
2,0
3,0
4,1
...,...
952,0
953,0
954,0
955,0


In [25]:
y.value_counts()

Spam
0       835
1       122
dtype: int64

# Implementing Stratified k Fold, since there is imbalance in dataset

# Implementing Logistic Regression

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB,GaussianNB

In [58]:
lr=LogisticRegression()
sv=SVC()
gb=GaussianNB()
mb=MultinomialNB()

In [26]:
from sklearn.model_selection import StratifiedKFold,train_test_split

In [60]:
skf=StratifiedKFold(n_splits=10,shuffle=True,random_state=1)
list_of_accuracy_lr=[]
list_of_accuracy_sv=[]
list_of_accuracy_gb=[]
list_of_accuracy_mb=[]

In [61]:
for train_index,test_index in skf.split(x,y):
    x_train_fold,x_test_fold=x[train_index],x[test_index]
    y_train_fold,y_test_fold=y.iloc[train_index],y.iloc[test_index]
    lr.fit(x_train_fold,y_train_fold)
    sv.fit(x_train_fold,y_train_fold)
    gb.fit(x_train_fold,y_train_fold)
    mb.fit(x_train_fold,y_train_fold)
    list_of_accuracy_lr.append(lr.score(x_test_fold,y_test_fold))
    list_of_accuracy_sv.append(sv.score(x_test_fold,y_test_fold))
    list_of_accuracy_gb.append(lr.score(x_test_fold,y_test_fold))
    list_of_accuracy_mb.append(sv.score(x_test_fold,y_test_fold))

C:\Users\Ratnesh\anaconda4\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Ratnesh\anaconda4\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Ratnesh\anaconda4\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Ratnesh\anaconda4\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  ret

In [52]:
from statistics import mean,stdev

# Parameters of logistic regression

In [62]:
print('List of possible accuracy for logistic regression:', list_of_accuracy_lr)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(list_of_accuracy_lr)*100, '%')
print('\nMinimum Accuracy:',
      min(list_of_accuracy_lr)*100, '%')
print('\nOverall Accuracy:',
      mean(list_of_accuracy_lr)*100, '%')
print('\nStandard Deviation is:', stdev(list_of_accuracy_lr))

List of possible accuracy for logistic regression: [0.8958333333333334, 0.8958333333333334, 0.9166666666666666, 0.8854166666666666, 0.9166666666666666, 0.90625, 0.8958333333333334, 0.8842105263157894, 0.8947368421052632, 0.8947368421052632]

Maximum Accuracy That can be obtained from this model is: 91.66666666666666 %

Minimum Accuracy: 88.42105263157895 %

Overall Accuracy: 89.86184210526315 %

Standard Deviation is: 0.011279265822226986


# Parameters of SVM 

In [63]:
print('List of possible accuracy for SVM:', list_of_accuracy_sv)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(list_of_accuracy_sv)*100, '%')
print('\nMinimum Accuracy:',
      min(list_of_accuracy_sv)*100, '%')
print('\nOverall Accuracy:',
      mean(list_of_accuracy_sv)*100, '%')
print('\nStandard Deviation is:', stdev(list_of_accuracy_sv))

List of possible accuracy for SVM: [0.9270833333333334, 0.9270833333333334, 0.9375, 0.9375, 0.9270833333333334, 0.96875, 0.9583333333333334, 0.9368421052631579, 0.9368421052631579, 0.9157894736842105]

Maximum Accuracy That can be obtained from this model is: 96.875 %

Minimum Accuracy: 91.57894736842105 %

Overall Accuracy: 93.72807017543859 %

Standard Deviation is: 0.015662562575148333


# Parameters of GuassianNb 

In [64]:
print('List of possible accuracy:', list_of_accuracy_gb)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(list_of_accuracy_gb)*100, '%')
print('\nMinimum Accuracy:',
      min(list_of_accuracy_gb)*100, '%')
print('\nOverall Accuracy:',
      mean(list_of_accuracy_gb)*100, '%')
print('\nStandard Deviation is:', stdev(list_of_accuracy_gb))

List of possible accuracy: [0.8958333333333334, 0.8958333333333334, 0.9166666666666666, 0.8854166666666666, 0.9166666666666666, 0.90625, 0.8958333333333334, 0.8842105263157894, 0.8947368421052632, 0.8947368421052632]

Maximum Accuracy That can be obtained from this model is: 91.66666666666666 %

Minimum Accuracy: 88.42105263157895 %

Overall Accuracy: 89.86184210526315 %

Standard Deviation is: 0.011279265822226986


# Parameters of MultinomialNb

In [65]:
print('List of possible accuracy:', list_of_accuracy_mb)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(list_of_accuracy_mb)*100, '%')
print('\nMinimum Accuracy:',
      min(list_of_accuracy_mb)*100, '%')
print('\nOverall Accuracy:',
      mean(list_of_accuracy_mb)*100, '%')
print('\nStandard Deviation is:', stdev(list_of_accuracy_mb))

List of possible accuracy: [0.9270833333333334, 0.9270833333333334, 0.9375, 0.9375, 0.9270833333333334, 0.96875, 0.9583333333333334, 0.9368421052631579, 0.9368421052631579, 0.9157894736842105]

Maximum Accuracy That can be obtained from this model is: 96.875 %

Minimum Accuracy: 91.57894736842105 %

Overall Accuracy: 93.72807017543859 %

Standard Deviation is: 0.015662562575148333
